# Протоколы обмена ключами

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

In [3]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

In [4]:
def randint(a, b):
    """
    Генерация слуайного числа среди заданных.
    """
    
    native_a = int(a)
    native_b = int(b)
    return gmpy2.mpz(random.randint(native_a, native_b))

----

In [5]:
# Генерация простых чисел и КГПСЧ
from Crypto.Util.number import getStrongPrime
from Crypto.Random import random

# HKDF и то, что надо для нее
from Crypto.Protocol.KDF import HKDF
from Crypto.Hash import SHA512

# Математическая библиотека
import gmpy2

# Для генерации идентификаторов
import uuid

# Утилитки
from Crypto.Util.number import long_to_bytes
import binascii

----

## Протокол Диффи-Хеллмана
![Diffe-Hellman Protocol](./images/Diffie-Hellman_4.png)

In [6]:
class DiffieHellmanParams(object):
    def __init__(self, prime_bits: int = 512):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[DiffieHellmanParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [7]:
class DiffieHellmanUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: DiffieHellmanParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        self._sabotage = use_another_value
        
        trace('[DiffieHellmanUser]', f'client {self._id} created')
    
    
    def initiate_key_exchange(self, another_client):
        """
        Инициирование протокола обмена ключами.
        """
        
        x = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{x = }')
        
        intermediate_value = another_client._accept_key_exchange(gmpy2.powmod(self._g, x, self._p))
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[DiffieHellmanUser]', f'[{self._id}]', f'guessed {x = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, x, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        
    def _accept_key_exchange(self, intermediate_value) -> int:
        y = self._generate_group_element()
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'{y = }')
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.powmod(intermediate_value, y, self._p))), 
                         SHA512, 
                         1)
        
        trace('[DiffieHellmanUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return gmpy2.powmod(self._g, y, self._p)
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [8]:
# Параметры протокола
params = DiffieHellmanParams()

[TRACE] [DiffieHellmanParams] p = 10428731291998092708225768852672661786629339248193303723612681284425535344362478914012810720143246807897693034607605324045531381675821013696990371554127783, g = 4301339014939559985474855587083996256097517546834736007476966918742915942379994122734930043514585558127336035141412422527939075072712681708015398568774242


In [9]:
# Два хороших пользователя
alice = DiffieHellmanUser(params)
bob   = DiffieHellmanUser(params)

[TRACE] [DiffieHellmanUser] client 46f76de0-26f2-4e5b-92d5-1cc4d3e96f68 created
[TRACE] [DiffieHellmanUser] client 2d2e4899-3f97-4ba7-b997-7f8f51f1269c created


In [10]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [46f76de0-26f2-4e5b-92d5-1cc4d3e96f68] x = mpz(3563546074257137441635687908081342434064504199780948208742471580783901289741424471608447646109999868299079469796526650460104602689029126141311470140093097)
[TRACE] [DiffieHellmanUser] [2d2e4899-3f97-4ba7-b997-7f8f51f1269c] y = mpz(8105488266005785821048334505001785599079297875163196965787612587164284575654462815382008140188579033373664017659521521733355002585065333643897301442165795)
[TRACE] [DiffieHellmanUser] [2d2e4899-3f97-4ba7-b997-7f8f51f1269c] derived key: 7b79b23172f87f85969a81827b2ae07d1503235380344dab668deb2312a7f705
[TRACE] [DiffieHellmanUser] [46f76de0-26f2-4e5b-92d5-1cc4d3e96f68] derived key: 7b79b23172f87f85969a81827b2ae07d1503235380344dab668deb2312a7f705


In [11]:
# Теперь врага создам
mallory = DiffieHellmanUser(params, True)

[TRACE] [DiffieHellmanUser] client fd4e48ec-e28a-49cd-93d3-31d7f060473e created


In [12]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [DiffieHellmanUser] [fd4e48ec-e28a-49cd-93d3-31d7f060473e] x = mpz(8968179488118087289954538311395873662968061651633718211385052277601352234902505873950517330178504791404873645691274618766005477396737689275411940240612664)
[TRACE] [DiffieHellmanUser] [2d2e4899-3f97-4ba7-b997-7f8f51f1269c] y = mpz(8377717705011089738069097518955561698039842930512726440314305514989519613052697212280191217093228934525319707581597183494623238665870668623422074115368343)
[TRACE] [DiffieHellmanUser] [2d2e4899-3f97-4ba7-b997-7f8f51f1269c] derived key: 1dc2b22ab4f4f12a70218e21a6205e0d74621c8d0b9d947f4394ebea5ab777f5
[TRACE] [DiffieHellmanUser] [fd4e48ec-e28a-49cd-93d3-31d7f060473e] guessed x = mpz(469846769549651978630648115746471375911785734977735446083514632172456159060305717254818633813155463115611482589078351975367907335007066793272628427506433)
[TRACE] [DiffieHellmanUser] [fd4e48ec-e28a-49cd-93d3-31d7f060473e] derived key: 32b7aa2268096fa6545a4f7ce29359509f9fc0f9f753684019cd5308486949be


----
## Протокол MTI
![Diffe-Hellman Protocol](./images/MTI_4.png)

In [13]:
class MTIParams(object):
    def __init__(self, prime_bits):
        """
        Генерация параметров протокола - простого числа и генератора.
        """
        
        self._p = gmpy2.mpz(getStrongPrime(prime_bits))
        self._g = randint(2, self._p - 2)
        
        trace('[MTIParams]', f'p = {self._p}, g = {self._g}')
        
        
    @property
    def prime(self) -> int:
        """
        Получение простого числе - модуля.
        """
        
        return self._p
    
    
    @property
    def generator(self) -> int:
        """
        Получение генератора.
        """
        
        return self._g

In [14]:
class MTIUser(object):
    KEY_LENGTH = 32
    HKDF_DUMMY_STUFF = b'\x00' * 16
    
    def __init__(self, params: MTIParams, 
                 use_another_value: bool = False):
        """
        Инициализация клиента: генерация идентификатора,
        сохранение параметров протокола, генерация ключевой пары.
        Последний параметр служит для демонстрации неуспешного выполнения.
        """
        
        self._p = params.prime
        self._g = params.generator
        self._id = uuid.uuid4()
        self._key = None
        
        self._private_key = self._generate_group_element()
        self._public_key  = gmpy2.powmod(self._g, self._private_key, self._p)
        
        self._sabotage = use_another_value
        
        trace('[MTIUser]', f'client {self._id} created. PK = {self._public_key}, SK = {self._private_key}')
        
        
    def initiate_key_exchange(self, anouther_client):
        """
        """
        
        x = self._generate_group_element()
        m = gmpy2.powmod(self._g, x, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{x = }, {m = }')
        
        intermediate_value = anouther_client._accept_key_exchange(m, self.public_key)
        
        #
        # Если требуют неудачную попытку, то подменю x
        # Это как бы незнание изначального значения - попытка угадать
        #
        
        if self._sabotage:
            x = self._generate_group_element()
            trace('[MTIUser]', f'[{self._id}]', f'guessed {x = }')
        
        product = gmpy2.powmod(anouther_client.public_key, x, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(DiffieHellmanUser.HKDF_DUMMY_STUFF, 
                         DiffieHellmanUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
    
    
    def _accept_key_exchange(self, intermediate_value, public_key):
        y = self._generate_group_element()
        m = gmpy2.powmod(self._g, y, self._p)
        
        trace('[MTIUser]', f'[{self._id}]', f'{y = }, {m = }')
        
        product = gmpy2.powmod(public_key, y, self._p) * \
            gmpy2.powmod(intermediate_value, self._private_key, self._p)
        
        self._key = HKDF(MTIUser.HKDF_DUMMY_STUFF, 
                         MTIUser.KEY_LENGTH, 
                         long_to_bytes(int(gmpy2.t_mod(product, self._p))), 
                         SHA512, 
                         1)
        
        trace('[MTIUser]', f'[{self._id}]', f'derived key: {bytes_as_hex(self._key)}')
        
        return m
        
    
    @property
    def public_key(self):
        """
        """
        
        return self._public_key
    
    
    def _generate_group_element(self) -> int:
        return randint(1, self._p - 2)

----

In [15]:
# Параметры протокола
params = MTIParams(512)

[TRACE] [MTIParams] p = 11592601963321632962173155328472750673145797612659734012251660127980411229708341027100040633763113430514552681592120166655048699037243311713770076804212167, g = 3255147961521296189001775336998802949170684672590815480191330616174841518951796532383163677053141407603193639832737827221839116285105764499671072245565986


In [16]:
# Два хороших пользователя
alice = MTIUser(params)
bob   = MTIUser(params)

[TRACE] [MTIUser] client 2b406b0f-299d-472f-bdfa-4ca3a1a68789 created. PK = 3345719960517728652840099437875777290200636612641785792685584703771882135531098721800091609592987112775476159546943352885808577467004648547843757867502356, SK = 9932574773053781585943476567790103941225736441089331620447554537609752971031388779273916622842440393532699405819159612610695198943956782669232377036313181
[TRACE] [MTIUser] client 8ab1986d-a089-4aa1-9a11-0784a54fef0f created. PK = 1539721278333445720750962334951312222097637887340834918826303036135222016690066555481517908401699111092533386027551856899483529853722298288197317269200984, SK = 1885853514046379598576240132423848455914739020666503888941069236718342021382321021193187133867715967026674192746226062700423639657637525050431639819541970


In [17]:
# Ключ получается установить успешно
alice.initiate_key_exchange(bob)

[TRACE] [MTIUser] [2b406b0f-299d-472f-bdfa-4ca3a1a68789] x = mpz(5195163145157225814881702735494209651833683412217767865237207710971649145915650585963427019296007059225391066300321546486014637521730172453063437966513920), m = mpz(939533131334524647779101630642529498646752308572206290756495261643809462468420913885971828987188934328476568038484574114255111894821352339214726508538993)
[TRACE] [MTIUser] [8ab1986d-a089-4aa1-9a11-0784a54fef0f] y = mpz(5196434234461629123025870737227303876880352408444336724086969340628043215378329202360064118689841843028542126232704347979074635975588622125712381774808052), m = mpz(6833547665946786041368698352270543714108133296692527712769788044428098935157133957862347399034226779687715303245490000515693002384347741478098934891004316)
[TRACE] [MTIUser] [8ab1986d-a089-4aa1-9a11-0784a54fef0f] derived key: 085a62d4b26e000248caaf70dd5c36d54333fcf0070c1612215fd2b40c783e9e
[TRACE] [MTIUser] [2b406b0f-299d-472f-bdfa-4ca3a1a68789] derived key: 085a62d4b26e000248caaf70

In [18]:
# Теперь врага создам
mallory = MTIUser(params, True)

[TRACE] [MTIUser] client 692e49b6-4680-4225-b0ad-ccb3c2307003 created. PK = 11344731106246299898403452740749014271591893957685700761575407163170231899018525037545860401669462403120262074267033365478746228012815008393235816395616398, SK = 10248664578219607675280212536177241694622751058862894646898537161781964094839948525056113094084804803522590461242487033562776779026580959994379392520997322


In [19]:
# Внедрившись в исполняющийся протокол, врагу не достается правильный ключ
mallory.initiate_key_exchange(bob)

[TRACE] [MTIUser] [692e49b6-4680-4225-b0ad-ccb3c2307003] x = mpz(5454933076143595521250291749260405352810867011545876613527588128550937112453022713898996451203060649510194177019772531819556660414833632093690183660176732), m = mpz(9136642007349247458154345204856822966885167845849148551433964567697925258176037715178295694941543531215385914198499662705562044370997697551429322310664617)
[TRACE] [MTIUser] [8ab1986d-a089-4aa1-9a11-0784a54fef0f] y = mpz(6071076336102888186511965737667619881447048968156353470857267685725062826618828055533950738167907654023111694008335410453854967665792824999216357015082642), m = mpz(8644208353450520482905366953483423609249567326869469984038369266432644132170426914323174014571535994167731036524133228744915161062804955499677898050890739)
[TRACE] [MTIUser] [8ab1986d-a089-4aa1-9a11-0784a54fef0f] derived key: dade286d329d84deba81bf5446ae01bfc852d4d7bbd4d130e1550af071c221af
[TRACE] [MTIUser] [692e49b6-4680-4225-b0ad-ccb3c2307003] guessed x = mpz(21743866647633529391